In [2]:
import os
import shutil
import glob
from tqdm import tqdm
import random

In [2]:
Raw_DIR= r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Dataset\Eye\mrlEyes_2018_01'
for dirpath, dirname, filenames in os.walk(Raw_DIR):
    for i in tqdm([f for f in filenames if f.endswith('.png')]):
        if i.split('_')[4]=='0':
            shutil.copy(src=dirpath+'/'+i, dst=r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Dataset\Closed Eyes')
        elif i.split('_')[4]=='1':
            shutil.copy(src=dirpath+'/'+i, dst=r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Dataset\Opened Eyes') 

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████| 10257/10257 [02:28<00:00, 68.86it/s]


In [3]:
Closed_Eyes_Dir = r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Dataset\Closed Eyes'
Closed_Eyes_Train_Dir = r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Dataset\Train\Closed Eyes'
Closed_Eyes_Test_Dir = r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Dataset\Test\Closed Eyes'

files = os.listdir(Closed_Eyes_Dir)
random.shuffle(files)

n_train = int(0.8 * len(files))
n_test = len(files) - n_train

for file in files[:n_train]:
    shutil.copy(os.path.join(Closed_Eyes_Dir, file), os.path.join(Closed_Eyes_Train_Dir, file))

for file in files[n_train:]:
    shutil.copy(os.path.join(Closed_Eyes_Dir, file), os.path.join(Closed_Eyes_Test_Dir, file))


In [ ]:
Opened_Eyes_Dir = r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Dataset\Opened Eyes'
Opened_Eyes_Train_Dir = r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Dataset\Train\Opened Eyes'
Opened_Eyes_Test_Dir = r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Dataset\Test\Opened Eyes'

files = os.listdir(Opened_Eyes_Dir)
random.shuffle(files)

n_train = int(0.8 * len(files))
n_test = len(files) - n_train

for file in files[:n_train]:
    shutil.copy(os.path.join(Opened_Eyes_Dir, file), os.path.join(Opened_Eyes_Train_Dir, file))

for file in files[n_train:]:
    shutil.copy(os.path.join(Opened_Eyes_Dir, file), os.path.join(Opened_Eyes_Test_Dir, file))

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)

train_data = train_datagen.flow_from_directory(r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Dataset\Train',
                                target_size=(80,80),batch_size=8,class_mode='categorical',subset='training' )

validation_data = train_datagen.flow_from_directory(r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Dataset\Train',
                                target_size=(80,80),batch_size=8,class_mode='categorical', subset='validation')

Found 54334 images belonging to 2 classes.
Found 13583 images belonging to 2 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Dataset\Test',
                                target_size=(80,80),batch_size=8,class_mode='categorical')

Found 16981 images belonging to 2 classes.


In [4]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 39, 39, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 39, 39, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 39, 39, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

__________________________________________________________________________________________________
activation_40 (Activation)      (None, 3, 3, 192)    0           batch_normalization_40[0][0]     
__________________________________________________________________________________________________
activation_43 (Activation)      (None, 3, 3, 192)    0           batch_normalization_43[0][0]     
__________________________________________________________________________________________________
activation_48 (Activation)      (None, 3, 3, 192)    0           batch_normalization_48[0][0]     
__________________________________________________________________________________________________
activation_49 (Activation)      (None, 3, 3, 192)    0           batch_normalization_49[0][0]     
__________________________________________________________________________________________________
mixed5 (Concatenate)            (None, 3, 3, 768)    0           activation_40[0][0]              
          

Total params: 21,934,050
Trainable params: 131,266
Non-trainable params: 21,802,784
__________________________________________________________________________________________________


In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [7]:
checkpoint = ModelCheckpoint(r'C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Models\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//8,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//8,
                   callbacks=callbacks,
                    epochs=1)

c:\users\mayur_mote\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


6791/6791 [==============================] - 867s 126ms/step - loss: 0.2079 - accuracy: 0.9185 - val_loss: 0.2362 - val_accuracy: 0.9043

Epoch 00001: val_loss improved from inf to 0.23620, saving model to C:\Users\Mayur_Mote\Documents\BE Project\Driver Drowsiness Detection System\Implementation\DDS Using NN\Models\model.h5


In [13]:
acc_test, loss_test = model.evaluate_generator(test_data)

In [14]:
print(acc_test)
print(loss_test)

0.1528148353099823
0.9410517811775208


In [16]:
pip install pygame

     ---------------------------------------- 10.6/10.6 MB 1.5 MB/s eta 0:00:00Note: you may need to restart the kernel to use updated packages.




[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
